In [ ]:
import numpy as np
from skimage import io
from utils import grainPreprocess, grainMark

from matplotlib import pyplot as plt

from skimage import io, color, filters, morphology, util
from skimage.measure import EllipseModel
from skimage.color import rgb2gray
from skimage import filters, util
from skimage.morphology import disk, skeletonize, ball
from skimage.measure import approximate_polygon
from skimage import transform
import copy
from PIL import Image, ImageDraw, ImageFilter, ImageOps

from matplotlib import cm

In [ ]:
def preprocess_image(image):
    if len(image.shape)==3:
        image = color.rgb2gray(image)

    image = filters.rank.median(image, morphology.disk(3))

    global_thresh = filters.threshold_otsu(image)
    image = image > global_thresh
    binary = image*255
    binary = binary.astype(np.uint8)

    grad = abs(filters.rank.gradient(binary, morphology.disk(1)))
    bin_grad = (1 - binary + grad) * 127
    bin_grad = np.clip(bin_grad, 0, 255).astype(np.uint8)

    return bin_grad

In [ ]:
def draw_edges(image, cnts, color=(0, 139, 139), r=4, e_width=5, l_width=4):

    img = copy.copy(image)
    draw = ImageDraw.Draw(img)

    for j, cnt in enumerate(cnts):
        if len(cnt) > 1:
            point = cnt[0]
            x1, y1 = point[1], point[0]

            for i, point2 in enumerate(cnt):
                p2 = point2

                x2, y2 = p2[1], p2[0]

                draw.ellipse((y2 - r, x2 - r, y2 + r, x2 + r), fill=color, width=e_width)
                draw.line((y1, x1, y2, x2), fill=(255, 140, 0), width=l_width)
                x1, y1 = x2, y2

    return img

In [ ]:
img = io.imread('../datasets/original/o_bc_left/Ultra_Co6_2/Ultra_Co6_2-001.jpeg')[:200,:200]
img = preprocess_image(img)

img_with_border = Image.fromarray(img)
img_with_border = ImageOps.expand(img_with_border,border=10, fill='white')

cnt=grainMark.get_contours(np.array(img_with_border),tol=3)

cnt_img=draw_edges(img_with_border.convert('RGB'),cnts=cnt, r=2, l_width=1)

# plt.figure(figsize=(10,10))
# plt.imshow(cnt_img)
# plt.show()

In [ ]:
img = io.imread('../datasets/original/o_bc_left/Ultra_Co6_2/Ultra_Co6_2-001.jpeg')[:200,:200]
img = preprocess_image(img)

r=2
eps = 5
border = 10
tol = 2

xs=[]
ys=[]

img_with_border = Image.fromarray(img)
img_with_border = ImageOps.expand(img_with_border, border=border, fill='white')

cnts=grainMark.get_contours(np.array(img_with_border),tol=tol)

for points in cnts:
    for point in points:
        if point[1]<10+eps:
            xs.append(point[1])
            ys.append(point[0])

img = copy.copy(img_with_border).convert('RGB')
img=draw_edges(img, cnts=cnts, r=2, l_width=1)
draw = ImageDraw.Draw(img)


for i in range(len(xs)):
    x = xs[i]
    y = ys[i]
    draw.ellipse((y - r, x - r, y + r, x + r), fill=(150,150,0), width=1)

plt.figure(figsize=(10,10))
plt.imshow(img,cmap='gray')
plt.show()
